In [ ]:
import re  #正規化
import math
import datetime as dt
import time 
import requests
from bs4 import BeautifulSoup as BS
import json
import ast  #轉換成json需要套件
how_much_crawl=1
year_2013={1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
#year_2014={ 1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
#year_2015={ 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
#year_2016={1: 31, 2: 29, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}

tic = time.clock()
failure_url_2013=[]
for mm in range(4,7):
    dd=year_2013[mm]
    print mm ,dd
    startdate = dt.datetime(2013,mm,1)
    endate = dt.datetime(2013,mm,dd)
    totaldays = (endate - startdate).days + 1 #期間有幾天
    count=0
    k=0
    all_json=[]
    
    for daynumber in range(totaldays):
        datestring = (startdate + dt.timedelta(days = daynumber)).date().strftime("%Y-%m-%d") 
        print datestring
        #每一天的蘋果新聞頁面
        #http://www.chinatimes.com/history-by-date/2016-03-19-2601?page=1 
        page_url = 'http://www.chinatimes.com/history-by-date/{}-2601?page=1'.format(datestring)
        #print page_url
        res_test=requests.get(page_url)
        soup_test=BS(res_test.text)
        total_pages1=soup_test.select('.listLeft ul')[0].select('li')[0].select('span')[1]
        #print total_pages1
        total_pages2=(str(total_pages1.text))
        lis=soup_test.select('.listRight li')
        how_many_new_in_page=len(lis)
        dic=[total_pages2,"apple"]
        for ele in dic: 
            #print dic
            m=re.search('\((?P<username>\d+)\)',ele)
            if m:
                total_pages=int(m.group('username'))   #總頁數
                #print total_pages

        how_many_page=int(math.ceil(total_pages/float(how_many_new_in_page))) #有多少分頁
        #print how_many_page

        #  以上得知 ↑↑↑   ------→ 每一天的新聞，各自有多少篇數，還有多少分頁 ---------------------------

        for j in xrange(how_many_page+1):#how_many_page+1
            secondPages='http://www.chinatimes.com/history-by-date/{}-2601?page={}'.format(datestring,j)  #僅此2016-03-19-2601，之後要做不同天的!
            #print "\n",secondPages  
            time.sleep(1.5)    #要睡喔! 不然的話會失敗，網頁沒開完整 就開始抓就會容易失敗!!!!!!!!!!!!!!!!!! Q_Q
        #  以上得知 ↑↑↑   ------→ 每一天的新聞不同的分頁數 ---------------------------
            url=secondPages
            res=requests.get(url)
            soup=BS(res.text)

            #print soup
            mainPage='http://www.chinatimes.com/'
            lis=soup.select('.listRight li')
            how_many_new_in_page=len(lis)
            l=0
            #print how_many_new_in_page
            for li in lis:
                #print l,"總頁數", how_much_crawl,"|",
                print how_much_crawl,
                how_much_crawl=how_much_crawl+1
                l=l+1
                if l<len(lis):   #單頁報導要寫入幾天 #len(lis)
                    title=li.select('h2')[0].text.strip() #list→unicode  加了[0]→'bs4.element.Tag' 可顯示中文字
                    #print title  #2016/3/23 ok
                    category=li.select('.kindOf a')[0].text.strip() #類別
                    #print category # 2016/3/23 ok
                    Data2=soup.select('.page_index li')[1]
                    Data1=Data2.select('h6 a')[0]['href'][0:].split('-')
                    Data=Data1[0]+Data1[1]+Data1[2]              #yyyymmdd  日期    
                    #print Data   # 2016/3/23 ok
                    newspaper_office='中國時報'                  #報社
                    page1=li.select('h2 a')[0]['href'][0:]      #新聞連結，進入到內文的階段
                    each_new_page_url=mainPage+page1 
                    #print each_new_page_url  # 2016/3/23 ok
                    second_url=requests.get(each_new_page_url)
                    second_usoup=BS(second_url.text)
                    #print len(second_usoup.findAll('div',{'class':'art_click clear-fix'})[0])
                    try:
                        title=second_usoup.select('header h1')[1]
                        #print title.text                    #title
                        if len(second_usoup.findAll('div',{'class':'art_click clear-fix'})[0])>1:

                            hits=second_usoup.findAll('div',{'class':'art_click clear-fix'})[0]
                            number_of_hits = int(hits.select('span')[1].text)            #點擊率
                            
                        else:
                            number_of_hits=0
                            failure_url_2013.append(each_new_page_url)
                            print "第幾分頁→", secondPages, "這一篇報導→",each_new_page_url
                    except IndexError:
                        print "沒有此網頁→" ,each_new_page_url                   #title
                        failure_url_2013.append(each_new_page_url)
                        continue    

                    all_p_tags=second_usoup.select('.clear-fix p')[:-6]
                    content_list=[]
                    for p_tag in all_p_tags:
                        p_tag1=str(p_tag).replace("，","_")
                        p_tag2=str(p_tag1).replace("。","_")
                        p_tag3=str(p_tag2).replace("<p>","")
                        p_tag4=str(p_tag3).replace("</p>","")
                        #print p_tag4    #2016/3/23 ok                         #內文(已將 "，" & "。"轉換)
                        content_list.append(p_tag4)                            #將全部 標籤 P寫入 list中
                    separator=''
                    joined_content=separator.join(content_list)              #將 list [i] 以 "" 合併，得到全文內容    |獨家秘技!!!!| 
                    #print joined_content  #2016/3/23 ok
                    keyword=second_usoup.select('.a_k a')
                    i=0
                    kwords_list=[]
                    for kwords in range(len(keyword)):
                        keyword=second_usoup.select('.a_k a')[i]
                        #print keyword.text                                       #此頁新聞的關鍵字
                        i=i+1
                        kwords_list.append(keyword.text)
                    separator='_'    
                    joined_kwords=separator.join(kwords_list)               #將 list [i] 以 "" 合併，得到關鍵字全部內容 
                    #print joined_kwords    #2016/3/23 ok
                    

                    m = {
                         'url':each_new_page_url,'title':title.text,'category':category,\
                         'content':joined_content,'date':Data,'hitcount':number_of_hits,\
                         'keyw':joined_kwords,'comp':'ChinaTimes'
                         }                          #填入json格式dict

                    #print m
                    all_json.append(m)            #dict不可以先加 所以先變STR相加   
                    time.sleep(1) 
                else:                           
                    break               



    toc = time.clock()
    print toc-tic
    #print m

 
    d1=json.dumps(all_json)
    #d=json.loads(d1)
    #with open('C:/Users/BIG DATA/pythonETL/project/news/20130{}.json'.format(mm), 'w') as f:
    with open('C:/Users/BIG DATA/pythonETL/project/news/2013{}.json'.format(mm), 'w') as f:
        f.write(d1) 
    test1=str(failure_url_2013)    
    with open('C:/Users/BIG DATA/pythonETL/project/news/failure_url_2013.txt', 'w') as f:
        f.write(test1)     
    #會有兩種 output!!!!!!   failure_news_2013.txt   &   20150{}.json

 4 30
2013-04-01
1 2 3 4 5 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=0 這一篇報導→ http://www.chinatimes.com//newspapers/20130401000363-260102
6 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=0 這一篇報導→ http://www.chinatimes.com//newspapers/20130401000366-260102
7 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=0 這一篇報導→ http://www.chinatimes.com//newspapers/20130401000368-260102
8 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=0 這一篇報導→ http://www.chinatimes.com//newspapers/20130401000369-260102
9 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=0 這一篇報導→ http://www.chinatimes.com//newspapers/20130401000370-260102
10 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=0 這一篇報導→ http://www.chinatimes.com//newspapers/20130401000371-260102
11 12 13 14 15 16 第幾分頁→ http://www.chinatimes.com/history-by-date/2013-04-01-2601?page=1 這一篇報導→ http://www.chinatimes.com//newspapers/

In [ ]:
print all_json

In [7]:
import json
d1=json.dumps(all_json)
with open('C:/Users/BIG DATA/pythonETL/project/news/2015{}.json'.format(mm), 'w') as f:
    f.write(d1) 
        

TypeError: <h1>\u9678\u88fd\u9020\u696dPMI\u90234\u5347</h1> is not JSON serializable

In [8]:
print type(all_json)

<type 'list'>
